In [ ]:
from pathlib import Path
import SimpleITK as sitk
from MeshMetrics.utils import create_synthetic_examples_2d, create_synthetic_examples_3d, vtk_write_polydata, sitk2np, vtk_read_polydata
from MeshMetrics import DistanceMetrics

# simple funciton to print results
def print_metrics(hd100, hd95, masd, assd, nsd2, biou2):
    print(f"HD100: {hd100:.2f} mm, HD (perc=95): {hd95:.2f} mm, MASD: {masd:.2f} mm, ASSD: {assd:.2f} mm, NSD (tau=2): {nsd2*100:.2f} %, BIoU (tau=2): {biou2*100:.2f} %")

In [ ]:
# Create synthetic examples (a few are already provided in the examples folder)
create_data = True
data_dir = Path('data')

if create_data:
    # 2D: Create synthetic circles with predefined radii
    ref_mesh, pred_mesh, ref_mask_sitk, pred_mask_sitk = create_synthetic_examples_2d(r1=34, r2=35.5, spacing=(0.2, 0.2))
    data_dir.mkdir(exist_ok=True)
    vtk_write_polydata(ref_mesh, data_dir / "example_2d_ref_mesh.obj")
    vtk_write_polydata(pred_mesh, data_dir / "example_2d_pred_mesh.obj")
    sitk.WriteImage(ref_mask_sitk, data_dir / "example_2d_ref_mask.nii.gz")
    sitk.WriteImage(pred_mask_sitk, data_dir / "example_2d_pred_mask.nii.gz")
    
    # 3D: Create synthetic spheres with predefined radii
    ref_mesh, pred_mesh, ref_mask_sitk, pred_mask_sitk = create_synthetic_examples_3d(r1=34, r2=35.5, spacing=(1.0, 1.0, 1.0))
    data_dir.mkdir(exist_ok=True)
    vtk_write_polydata(ref_mesh, data_dir / "example_3d_ref_mesh.obj")
    vtk_write_polydata(pred_mesh, data_dir / "example_3d_pred_mesh.obj")
    sitk.WriteImage(ref_mask_sitk, data_dir / "example_3d_ref_mask.nii.gz")
    sitk.WriteImage(pred_mask_sitk, data_dir / "example_3d_pred_mask.nii.gz")

2024-11-21 08:22:48.804 (   0.175s) [    7F4CBE6C1080]vtkPolyDataPlaneCutter.:589   INFO| Executing vtkPolyData plane cutter
2024-11-21 08:22:48.804 (   0.176s) [    7F4CBE6C1080]vtkPolyDataPlaneCutter.:589   INFO| Executing vtkPolyData plane cutter


# 3D examples

## Calculate metrics using `SimpleITK.Image` masks

In [ ]:
ref_mask_sitk = sitk.ReadImage(data_dir / "example_3d_ref_mask.nii.gz")
pred_mask_sitk = sitk.ReadImage(data_dir / "example_3d_pred_mask.nii.gz")
# print some metadata
ref_mask_sitk.GetSize(), ref_mask_sitk.GetSpacing()

((81, 81, 81), (1.0, 1.0, 1.0))

In [4]:
# Calculate metrics using SimpleITK masks
dist_metrics = DistanceMetrics()
# note that spacing is automatically inferred from the sitk.Image object
dist_metrics.set_input(ref=ref_mask_sitk, pred=pred_mask_sitk)

# Hausdorff Distance (HD), by default, HD percentile is set to 100 (equivalent to HD)
hd100 = dist_metrics.hd()
# 95th percentile HD
hd95 = dist_metrics.hd(percentile=95)
# Mean Average Surface Distance (MASD)
masd = dist_metrics.masd()
# Average Symmetric Surface Distance (ASSD)
assd = dist_metrics.assd()
# Normalized Surface Distance (NSD) with tau=2
nsd2 = dist_metrics.nsd(tau=2)
# Boundary Intersection over Union (BIoU) with tau=2
biou2 = dist_metrics.biou(tau=2)

print_metrics(hd100, hd95, masd, assd, nsd2, biou2)

HD100: 2.12 mm, HD (perc=95): 1.83 mm, MASD: 1.38 mm, ASSD: 1.38 mm, NSD (tau=2): 99.73 %, BIoU (tau=2): 34.34 %


## Calculate metrics using `numpy.ndarray` masks

In [5]:
# convert sitk to numpy
spacing = ref_mask_sitk.GetSpacing()
ref_mask_np = sitk2np(ref_mask_sitk) > 0
pred_mask_np = sitk2np(pred_mask_sitk) > 0
# check masks size and spacing
ref_mask_np.shape, pred_mask_np.shape, spacing

((81, 81, 81), (81, 81, 81), (1.0, 1.0, 1.0))

In [6]:
# Calculate metrics using SimpleITK masks
dist_metrics = DistanceMetrics()
# note that spacing is automatically inferred from the sitk.Image object
dist_metrics.set_input(ref=ref_mask_np, pred=pred_mask_np, spacing=spacing)

# Hausdorff Distance (HD), by default, HD percentile is set to 100 (equivalent to HD)
hd100 = dist_metrics.hd()
# 95th percentile HD
hd95 = dist_metrics.hd(percentile=95)
# Mean Average Surface Distance (MASD)
masd = dist_metrics.masd()
# Average Symmetric Surface Distance (ASSD)
assd = dist_metrics.assd()
# Normalized Surface Distance (NSD) with tau=2
nsd2 = dist_metrics.nsd(tau=2)
# Boundary Intersection over Union (BIoU) with tau=2
biou2 = dist_metrics.biou(tau=2)

print_metrics(hd100, hd95, masd, assd, nsd2, biou2)

HD100: 2.12 mm, HD (perc=95): 1.83 mm, MASD: 1.38 mm, ASSD: 1.38 mm, NSD (tau=2): 99.50 %, BIoU (tau=2): 34.34 %


## Calculate metrics using `vtk` meshes

In [ ]:
# load meshes
ref_mesh = vtk_read_polydata(data_dir / "example_3d_ref_mesh.obj")
pred_mesh = vtk_read_polydata(data_dir / "example_3d_pred_mesh.obj")
# set spacing to arbitrary value (this is only needed for BIoU calculation, the low the better, but it prolongs the computation)
spacing = (0.5, 0.5, 0.5)

In [8]:
# Calculate metrics using SimpleITK masks
dist_metrics = DistanceMetrics()
# note that spacing is automatically inferred from the sitk.Image object
dist_metrics.set_input(ref=ref_mesh, pred=pred_mesh, spacing=spacing)

# Hausdorff Distance (HD), by default, HD percentile is set to 100 (equivalent to HD)
hd100 = dist_metrics.hd()
# 95th percentile HD
hd95 = dist_metrics.hd(percentile=95)
# Mean Average Surface Distance (MASD)
masd = dist_metrics.masd()
# Average Symmetric Surface Distance (ASSD)
assd = dist_metrics.assd()
# Normalized Surface Distance (NSD) with tau=2
nsd2 = dist_metrics.nsd(tau=2)
# Boundary Intersection over Union (BIoU) with tau=2
biou2 = dist_metrics.biou(tau=2)

print_metrics(hd100, hd95, masd, assd, nsd2, biou2)

HD100: 1.50 mm, HD (perc=95): 1.50 mm, MASD: 1.50 mm, ASSD: 1.50 mm, NSD (tau=2): 100.00 %, BIoU (tau=2): 26.23 %


# 2D examples

## Calculate metrics using `SimpleITK.Image` masks

In [ ]:
ref_mask_sitk = sitk.ReadImage(data_dir / "example_2d_ref_mask.nii.gz")
pred_mask_sitk = sitk.ReadImage(data_dir / "example_2d_pred_mask.nii.gz")
# print some metadata
ref_mask_sitk.GetSize(), ref_mask_sitk.GetSpacing()

((365, 365), (0.20000000298023224, 0.20000000298023224))

In [10]:
# Calculate metrics using SimpleITK masks
dist_metrics = DistanceMetrics()
# note that spacing is automatically inferred from the sitk.Image object
dist_metrics.set_input(ref=ref_mask_sitk, pred=pred_mask_sitk)

# Hausdorff Distance (HD), by default, HD percentile is set to 100 (equivalent to HD)
hd100 = dist_metrics.hd()
# 95th percentile HD
hd95 = dist_metrics.hd(percentile=95)
# Mean Average Surface Distance (MASD)
masd = dist_metrics.masd()
# Average Symmetric Surface Distance (ASSD)
assd = dist_metrics.assd()
# Normalized Surface Distance (NSD) with tau=2
nsd2 = dist_metrics.nsd(tau=2)
# Boundary Intersection over Union (BIoU) with tau=2
biou2 = dist_metrics.biou(tau=2)

print_metrics(hd100, hd95, masd, assd, nsd2, biou2)

HD100: 1.60 mm, HD (perc=95): 1.57 mm, MASD: 1.46 mm, ASSD: 1.46 mm, NSD (tau=2): 100.00 %, BIoU (tau=2): 19.01 %


## Calculate metrics using `numpy.ndarray` masks

In [11]:
# convert sitk to numpy
spacing = ref_mask_sitk.GetSpacing()
ref_mask_np = sitk2np(ref_mask_sitk) > 0
pred_mask_np = sitk2np(pred_mask_sitk) > 0
# check masks size and spacing
ref_mask_np.shape, pred_mask_np.shape, spacing

((365, 365), (365, 365), (0.20000000298023224, 0.20000000298023224))

In [12]:
# Calculate metrics using SimpleITK masks
dist_metrics = DistanceMetrics()
# note that spacing is automatically inferred from the sitk.Image object
dist_metrics.set_input(ref=ref_mask_np, pred=pred_mask_np, spacing=spacing)

# Hausdorff Distance (HD), by default, HD percentile is set to 100 (equivalent to HD)
hd100 = dist_metrics.hd()
# 95th percentile HD
hd95 = dist_metrics.hd(percentile=95)
# Mean Average Surface Distance (MASD)
masd = dist_metrics.masd()
# Average Symmetric Surface Distance (ASSD)
assd = dist_metrics.assd()
# Normalized Surface Distance (NSD) with tau=2
nsd2 = dist_metrics.nsd(tau=2)
# Boundary Intersection over Union (BIoU) with tau=2
biou2 = dist_metrics.biou(tau=2)

print_metrics(hd100, hd95, masd, assd, nsd2, biou2)

HD100: 1.60 mm, HD (perc=95): 1.57 mm, MASD: 1.46 mm, ASSD: 1.46 mm, NSD (tau=2): 100.00 %, BIoU (tau=2): 19.01 %


## Calculate metrics using `vtk` meshes

In [ ]:
# load meshes
ref_mesh = vtk_read_polydata(data_dir / "example_2d_ref_mesh.obj")
pred_mesh = vtk_read_polydata(data_dir / "example_2d_pred_mesh.obj")
# set spacing to arbitrary value (this is only needed for BIoU calculation, the low the better, but it prolongs the computation)
spacing = (0.1, 0.1)

In [20]:
# Calculate metrics using SimpleITK masks
dist_metrics = DistanceMetrics()
# note that spacing is automatically inferred from the sitk.Image object
dist_metrics.set_input(ref=ref_mesh, pred=pred_mesh, spacing=spacing)

# Hausdorff Distance (HD), by default, HD percentile is set to 100 (equivalent to HD)
hd100 = dist_metrics.hd()
# 95th percentile HD
hd95 = dist_metrics.hd(percentile=95)
# Mean Average Surface Distance (MASD)
masd = dist_metrics.masd()
# Average Symmetric Surface Distance (ASSD)
assd = dist_metrics.assd()
# Normalized Surface Distance (NSD) with tau=2
nsd2 = dist_metrics.nsd(tau=2)
# Boundary Intersection over Union (BIoU) with tau=2
biou2 = dist_metrics.biou(tau=2)

print_metrics(hd100, hd95, masd, assd, nsd2, biou2)

HD100: 1.50 mm, HD (perc=95): 1.50 mm, MASD: 1.50 mm, ASSD: 1.50 mm, NSD (tau=2): 100.00 %, BIoU (tau=2): 16.86 %
